In [ ]:
import sys; sys.path.append('../../utils')
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns; sns.set_theme(style='darkgrid')

from utils import save_plot

In [ ]:
MONK_TASK = 2

In [ ]:
TRAIN_DATA = os.path.join('..', '..', '..', 'datasets', 'monk', f'monks-{MONK_TASK}.train')
TEST_DATA = os.path.join('..', '..', '..', 'datasets', 'monk', f'monks-{MONK_TASK}.test')
IMAGES_FOLDER = os.path.join('..', '..', '..', 'images', 'monk', f'task-{MONK_TASK}', 'neural_network')
MODEL_FOLDER = os.path.join('..', '..', '..', 'trained_models', 'monk', f'task-{MONK_TASK}')

In [ ]:
# To skip the first column (row indexes)
columns_to_read = list(range(1, 8))

df_train = pd.read_csv(TRAIN_DATA, header=None, usecols=columns_to_read, delimiter=' ')
df_test = pd.read_csv(TEST_DATA, header=None, usecols=columns_to_read, delimiter=' ')
df_train.head()

In [ ]:
features = ['feature_' + str(i) for i in range(1, 7)]

# Rename columns
new_column_names = ['class'] + features

df_train.columns = new_column_names
df_test.columns = new_column_names

df_train.head()

In [ ]:
df_train_encoded = pd.get_dummies(df_train, columns=features)
df_test_encoded = pd.get_dummies(df_test, columns=features)

df_train_encoded, df_test_encoded = df_train_encoded.align(df_test_encoded, join='inner', axis=1)

df_train_encoded.head()

In [ ]:
features = df_train_encoded.columns.difference(['class'])

X_train = df_train_encoded[features].to_numpy()
y_train = df_train_encoded['class'].to_numpy()

X_test = df_test_encoded[features].to_numpy()
y_test = df_test_encoded['class'].to_numpy()

# Create model

In [ ]:
from keras.src.regularizers import l2
from keras.src.optimizers import Adam, SGD
from keras.models import Sequential
from keras.layers import Dense

def create_model(architecture=(8,), optimizer='adam', learning_rate=0.001, lambda_value=0.1, momentum=0.9):
    model = Sequential()
    model.add(Dense(units=architecture[0], input_dim=X_train.shape[1], activation='relu',
                    kernel_regularizer=l2(lambda_value)))

    for units in architecture[1:]:
        model.add(Dense(units=units, activation='sigmoid', kernel_regularizer=l2(lambda_value)))

    model.add(Dense(1, activation='sigmoid'))

    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate, beta_1=momentum)
    else:
        opt = SGD(learning_rate=learning_rate, momentum=momentum)

    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model


In [ ]:
from scikeras.wrappers import KerasClassifier

model = KerasClassifier(create_model, verbose=0)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'model__architecture': [
        (8,), (8, 8), (8, 8, 8)
    ],
    'model__optimizer': ['sgd'],
    'model__learning_rate': [0.1, 0.3, 0.5],
    'model__momentum': [0.6, 0.8, 0.9],
    'model__lambda_value': [0, 0.01],
    'epochs': [100],
    'batch_size': [8]
}

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    verbose=2,
    scoring='accuracy'
)

grid_search.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

final_model = grid_search.best_estimator_
y_pred = final_model.predict(X_test)

print('Best parameters: ', grid_search.best_params_)
print('Best accuracy: ', grid_search.best_score_)
print('Test set accuracy: ', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Learning curve

In [ ]:
best_params = grid_search.best_params_

best_model = create_model(architecture=best_params['model__architecture'],
                          optimizer=best_params['model__optimizer'],
                          learning_rate=best_params['model__learning_rate'],
                          lambda_value=best_params['model__lambda_value'],
                          momentum=best_params['model__momentum'])

history = best_model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=0, validation_split=0.1)

In [ ]:
pd.DataFrame(history.history).plot(
    figsize=(8, 5),
    xlim=[0, 29],
    ylim=[0, 1],
    grid=True,
    xlabel='Epoch',
    style=['r--', 'r--.', 'b-', 'b-*']
)

plt.show()

# Save model

In [ ]:
from joblib import dump

model_path = os.path.join(MODEL_FOLDER, 'NN_model.joblib')
dump(final_model, model_path, compress=3)